<a href="https://colab.research.google.com/github/yukinaga/ai_programming/blob/main/lecture_09/04_distributed_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分散表現の確認
word2vecによる分散表現について学びます。

## ライブラリのインストール
必要なライブラリをインストールします。

In [ ]:
!pip install janome
!pip install gensim==4.1.2

## コーパスの前処理
コーパスに前処理を行います。

In [ ]:
import re
import pickle
from janome.tokenizer import Tokenizer

with open("wagahaiwa_nekodearu.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    wagahai_original = f.read()

wagahai = re.sub("《[^》]+》", "", wagahai_original) # ルビの削除
wagahai = re.sub("［[^］]+］", "", wagahai) # 読みの注意の削除
wagahai = re.sub("[｜ 　「」\n]", "", wagahai) # | と全角半角スペース、「」と改行の削除

seperator = "。"  # 。をセパレータに指定
wagahai_list = wagahai.split(seperator)  # セパレーターを使って文章をリストに分割する
wagahai_list.pop() # 最後の要素は空の文字列になるので、削除
wagahai_list = [x+seperator for x in wagahai_list]  # 文章の最後に。を追加
        
t = Tokenizer()

wagahai_words = []
for sentence in wagahai_list:
    wagahai_words.append(list(t.tokenize(sentence, wakati=True)))  # 文章ごとに単語に分割し、リストに格納
    
with open('wagahai_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(wagahai_words, f)

保存できていることを確認します。

In [ ]:
with open('wagahai_words.pickle', mode='rb') as f:
    wagahai_words = pickle.load(f)

print(wagahai_words[:10])

## word2vecを用いた学習
今回はword2vecのためにライブラリgensimを使います。  
gensimは、様々なトピックモデルを実装したPythonライブラリです。  
トピックモデルとは、潜在的なトピックから文章が確率的に生成されると仮定したモデルです。

gensimについて、詳細は以下のリンクを参考にどうぞ。  
https://radimrehurek.com/gensim/

以下では、word2vecを用いてコーパスの学習を行い、学習済みのモデルを作成します。

In [ ]:
from gensim.models import word2vec

# vector_size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# epochs : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          vector_size=100,
                          min_count=5,
                          window=5,
                          epochs=20,
                          sg = 0)

分散表現を見ていきましょう。  
重みを表す行列（分散表現）の形状と、行列そのものを表示します。

In [ ]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

語彙の数、および語彙の最初の10語を表示します。

In [ ]:
print(len(model.wv.index_to_key))  # 語彙の数
print(model.wv.index_to_key[:10])  # 最初の10単語

語彙における、最初の単語の単語ベクトルを2通りの方法で表示します。

In [ ]:
print(model.wv.vectors[0])  # 最初のベクトル
print(model.wv.__getitem__("の"))  # 最初の単語「の」のベクトル

両者ともに同じベクトルですね。

## 課題:
単語「猫」を単語ベクトルで表してみましょう。  
また、skip-gramも試して結果を比較してみましょう。